In [1]:
# webdriver-manager 불러오기
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()

'/Users/mac/.wdm/drivers/chromedriver/mac64/121.0.6167.85/chromedriver-mac-x64/chromedriver'

#### chromedriver 실행이 되지 않을경우 경로지정: https://emessell.tistory.com/entry/PYTHON-selenium-webdriver-%EA%B2%BD%EB%A1%9C-%EC%B2%98%EB%A6%AC

- (1) 상대경로
path = "./chromedriver"

- (2) 절대경로
import os
path = os.getcwd() + "/chromedriver"

- browser = webdriver.Chrome() # 경로지정
- browser.get("http://naver.com")

# 기본함수
- browser.back() # 뒤로가기
- browser.forward() # 앞으로가기
- browser.refresh() # 새로고침

# 문법
- By.CLASS_NAME	태그의 클래스명으로 추출
- By.TAG_NAME	태그 이름으로 추출
- By.ID	태그의 id값으로 추출
- By.XPATH	태그의 경로로 추출


In [22]:
# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

import time
import random

In [17]:
# 브라우저 띄우기
options = Options()

user = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
options.add_argument(f"User-Agent = {user}")
browser = webdriver.Chrome(options = options)
# 원하는 url로 이동
url = "https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24"
browser.get(url)

In [ ]:
# 1페이지 링크 데이터 전부 수집
links = browser.find_elements(By.CLASS_NAME , "gd_name")
print(len(links))
link_list = []
for link in links:
    link_list.append(link.get_attribute("href"))

In [25]:
# 브라우저 띄우기
options = Options()
user = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
options.add_argument(f"User-Agent = {user}")
browser = webdriver.Chrome(options=options)

link_list = []
for page in range(1,4):
    print("*"*10, f'현재 {page} 페이지 수집중입니다.', "*"*10)
    url = "https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={}&pageSize=24".format(page)
    
    # 브라우저 띄우기
    browser.get(url)
     # 각 페이지마다 도서의 링크 데이터 수집
    data = browser.find_elements(By.CLASS_NAME, 'gd_name')
    for i in data:
        link = i.get_attribute("href")
        link_list.append(link)

    time.sleep(3)

print(link_list)

********** 현재 1 페이지 수집중입니다. **********
********** 현재 2 페이지 수집중입니다. **********
********** 현재 3 페이지 수집중입니다. **********
['https://www.yes24.com/Product/Goods/122120495', 'https://www.yes24.com/Product/Goods/123675187', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/124043812', 'https://www.yes24.com/Product/Goods/124043802', 'https://www.yes24.com/Product/Goods/122090360', 'https://www.yes24.com/Product/Goods/123400303', 'https://www.yes24.com/Product/Goods/122944685', 'https://www.yes24.com/Product/Goods/124702623', 'https://www.yes24.com/Product/Goods/124112599', 'https://www.yes24.com/Product/Goods/118579613', 'https://www.yes24.com/Product/Goods/124574394', 'https://www.yes24.com/Product/Goods/124501342', 'https://www.yes24.com/Product/Goods/124539805', 'https://www.yes24.com/Product/Goods/119562091', 'https://www.yes24.com/Product/Goods/124408170', 'https://www.yes24.com/Product/Goods/122428190', 'https://www.yes24.com/Product/Goods/119458052', 